In [13]:
import pandas as pd
data = pd.read_csv("twitter_labeled_dataframe.csv")
data = data[data["ranges"].isna() == False]

In [14]:
data

,Unnamed: 0,sentence,labels,ranges
9,9,you can apply for compensation here if you had...,josh,"[[99, 103]]"
10,10,we'd be happy to look into this issue for you....,lindsey,"[[100, 107]]"
13,13,hi andrew. this is great to hear. which train ...,"andrew, josh","[[3, 9], [116, 120]]"
16,16,nora culik,nora culik,"[[0, 10]]"
18,18,we request you to share your alternate contact...,sneha,"[[117, 122]]"
...,...,...,...,...
928,928,"hey michael, we appreciate your feedback! plea...",michael,"[[4, 11]]"
931,931,"our sincerest apologies, dave. be sure to reac...",dave,"[[25, 29]]"
939,939,"hi chunhua, we have just responded to your dm ...",chunhua,"[[3, 10]]"
942,942,"oh no, jeff. sorry to hear about this terrible...",jeff,"[[7, 11]]"


In [15]:
training_data = []

import json

training_test_text = []

training_test_entity = []

for index, row in data.iterrows():
    text = row["sentence"]
    range_word = json.loads(row["ranges"])
    temp_list = []
    
    if len(range_word) == 1:
        range_word = range_word[0]
        range_word.append("PER_CUSTOM")
        entity_dict ={"entities" : [tuple(range_word)]}
    else:
        for i, inner_range in enumerate(range_word):
            inner_range.append("PER_CUSTOM")
            temp_list.append(tuple(inner_range))
        range_word = temp_list
        entity_dict = {"entities" : tuple(range_word)}
    
    training_test_text.append(text)

    training_test_entity.append(entity_dict)
        
    
    

In [16]:
from sklearn.model_selection import train_test_split

train_text, test_text, train_entity, test_entity = train_test_split(training_test_text, 
                                                                   training_test_entity, 
                                                                   test_size=0.2, shuffle=False)

In [17]:
search_for = "{'entities': ((16, 19, 'PER_CUSTOM'), (117, 120, 'PER_CUSTOM'))}"
for i, labels in enumerate(train_entity):
    if str(labels) == search_for:
        print("Found at", i)

In [18]:
train_text[68]

"you're welcome, ken! we realize that buffering can be a frustrating thing to endure, so no worries. have a great weekend! 💚"

In [1]:
import spacy
print(spacy.__version__)
from spacy.training import Example

def create_pipeline():
    model = None
    if model is not None:
        nlp = spacy.load(model)
        print("Loads model '%s'" % model)
    else:
        nlp = spacy.blank("en")
        print("Created blank 'en' model")
    if 'ner' not in nlp.pipe_names:
        ner = nlp.add_pipe("ner")
    else:
        print("get pipeline")
        ner = nlp.get_pipe("ner")
    return ner, nlp

3.5.1


In [20]:
ner, nlp = create_pipeline()
n_iter = 3
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(other_pipes):
    optimizer = nlp.begin_training()
for itn in range(n_iter):
    losses = {}
    for text, annotations in zip(train_text, train_entity):
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], losses=losses)
        
    print("Iteration", itn + 1, "Losses", losses)

Created blank 'en' model


C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "you're welcome, ken! we realize that buffering can..." with entities "((16, 19, 'PER_CUSTOM'), (116, 119, 'PER_CUSTOM'))". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Iteration 1 Losses {'ner': 203.36983752432224}
Iteration 2 Losses {'ner': 22.326144694480472}
Iteration 3 Losses {'ner': 9.826959182029421}


In [21]:
import pickle
with open("./models/custom_model_twitter.pickel", "wb") as f:
    pickle.dump(nlp, f)

In [22]:
df_test = pd.DataFrame(columns=["sentence", "label"])

df_test["sentence"] = test_text
df_test["label"] = test_entity

df_test.to_csv("test_data.csv")

In [23]:
from spacy.scorer import Scorer

# Default scoring pipeline
scorer = Scorer()


In [1]:
class NERmodel:
    def __init__(self, data):
        self.data = data
        self.nlp = None
        
    def __create_pipeline():
        model = None
        if model is not None:
            nlp = spacy.load(model)
            print("Loads model '%s'" % model)
        else:
            nlp = spacy.blank("en")
            print("Created blank 'en' model")
        if 'ner' not in nlp.pipe_names:
            ner = nlp.add_pipe("ner")
        else:
            print("get pipeline")
            ner = nlp.get_pipe("ner")
        return ner, nlp
        
    def train(epochs):
        ner, nlp = __create_pipeline()
        n_iter = epochs
        other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
        with nlp.disable_pipes(other_pipes):
            optimizer = nlp.begin_training()
        for itn in range(n_iter):
            losses = {}
            for text, annotations in zip(train_text, train_entity):
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], losses=losses)

            print("Iteration", itn + 1, "Losses", losses)
            
    def save_model(modelpath):
        with open(modelpath, "wb") as f:
            pickle.dump(nlp, f)
    
    def load_model(modelpath):
        with open(modelpath, "rb") as f:
            nlp = pickle.load(f)